# Exploratory Data Analysis for Car Insurance Fraud

The following report will use an open-source dataset uploaded by Oracle. This data sets includes  more then 15000 data samples about insurance claims, they include really important variables that could be correlated with the target variable of fraudulent or non-fraudulent claims.